En este sencillo cuaderno, usamos una red neuronal completamente conectada para resolver un problema visto anteriormente en regresión: el problema del corrimiento al rojo fotométrico.

Acompaña al Capítulo 8 del libro.

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.Traducido por Lucia Perez y Rosario Cecilio-Flores-Elie. 

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams['figure.dpi'] = 300

Tensorflow es una biblioteca muy utilizada en el desarrollo de modelos de aprendizaje profundos. Es una plataforma de código abierto desarrollada por Google. Admite la programación en varios lenguajes, p. C++, Java, Python y muchos otros.

Keras es una API (interfaz de programación de aplicaciones) de alto nivel que se basa en TensorFlow (o Theano, otra biblioteca de aprendizaje profundo). Es específico de Python y podemos considerarlo como el equivalente de la biblioteca sklearn para redes neuronales. Es menos general y menos personalizable, pero es muy fácil de usar y comparativamente más fácil que TensorFlow. Usaremos keras con el back-end de tensorflow.

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
import keras

from keras.models import Sequential #el modelo se construye añadiendo capas una tras otra

from keras.layers import Dense #capas totalmente conectadas: cada salida habla con cada entrada

from keras.layers import Dropout #para la regularización

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

### Problema 2: corrimientos al rojo fotométricos




Comenzaré con el conjunto de datos reducido (de alta calidad) que usamos para los métodos de Boosting y Bagging. Como referencia, nuestro mejor modelo logró un NMAD de alrededor de 0,02 y una fracción atípica del 4 %.

In [ ]:
X = pd.read_csv('../data/sel_features.csv', sep = '\t')
y = pd.read_csv('../data/sel_target.csv')

In [ ]:
X,y = shuffle(X,y, random_state = 12)

In [ ]:
fifth = int(len(y)/5)

In [ ]:
X_train = X.values[:3*fifth,:]
y_train = y[:3*fifth]

X_val = X.values[3*fifth:4*fifth,:]
y_val = y[3*fifth:4*fifth]

X_test = X.values[4*fifth:,:]
y_test = y[4*fifth:]

¡Sabemos que necesitamos escalar!

In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

In [ ]:
Xst_train = scaler.transform(X_train)
Xst_val = scaler.transform(X_val)
Xst_test = scaler.transform(X_test)

En un problema de regresión, elegiremos una activación diferente para la capa de salida (por ejemplo, lineal) y una función de pérdida diferente (MSE, MAE, ...).

Nuestra capa de entrada tiene seis neuronas para este problema.

Para otros parámetros y la estructura de la red, podemos comenzar con dos capas con 100 neuronas e ir desde allí.

In [ ]:
dir(keras.activations)

In [ ]:
dir(keras.losses)

In [ ]:
model = Sequential()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Agregue una capa de entrada y especifique su tamaño (número de características originales)

model.add(Dense(100, activation='relu', input_shape=(6,)))

#model.add(Dropout(0.2))

# Agregue una capa oculta y especifique su tamaño

model.add(Dense(100, activation='relu'))

#model.add(Dropout(0.2))

# Agregue una capa oculta y especifique su tamaño

#model.add(Dense(30, activation='relu'))

# Agregue una capa oculta y especifique su tamaño

#model.add(Dense(12, activation='relu'))

#model.add(Dropout(0.2))

# Agregar una capa de salida

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer=optimizer)


Comenzamos con 100 épocas y tamaño de lote = 300.

In [ ]:
mynet = model.fit(Xst_train, y_train, validation_data= (Xst_val, y_val), epochs=100, batch_size=300)

In [ ]:
results = model.evaluate(Xst_test, y_test)
print('MSE:', results) #solo estamos monitoreando MSE

As usual, we can plot the loss.

In [ ]:
plt.plot(mynet.history['loss'], label = 'train')
plt.plot(mynet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(loc='upper right', fontsize = 12)
plt.legend(fontsize = 12);
#plt.savefig('Photoz_NN.png')

In [ ]:
plt.figure(figsize=(5,5))
    
plt.xlabel('True redshift', fontsize = 14)
plt.ylabel('Estimated redshift', fontsize = 14)

plt.scatter(y_test, model.predict(Xst_test), s =10, c = 'teal');

plt.xlim(0,2)
plt.ylim(0,2)
plt.tight_layout()
#plt.savefig('Photoz_NN_scatter.png')

In [ ]:
ypred = model.predict(Xst_test)

### Revisión de aprendizaje
    
Calcule la fracción atípica y la desviación absoluta de la mediana normalizada para este conjunto de predicciones.

<br>

<details>
<summary style="display: line-item;">Haga clic aquí para la respuesta!</summary>
<p>
    
```python
print(len(np.where(np.abs(y_test-ypred)>0.15*(1+y_test))[0])/len(y_test))

print(1.48*np.median(np.abs(y_test-ypred)/(1 + y_test)))
```

</p>
</details>

Para mejorar aún más, podemos jugar con/optimizar los parámetros; una cosa que es muy interesante en mi opinión es ver el efecto de usar diferentes pérdidas en los residuos e intentar agregar más capas.

### Probemos algo de optimización con keras tuner

In [ ]:
# !pip3 install keras-tuner --upgrade    #si hace falta

In [ ]:
from keras_tuner.tuners import RandomSearch

from tensorflow.keras import layers

# Parte del material a continuación está adaptado de la documentación de Keras Tuner

# https://keras-team.github.io/keras-tuner/

Esta función especifica cuáles parámetros queremos ajustar. Los parámetros ajustables pueden ser del tipo "Choice" (especificamos un conjunto), Int, Boolean o Float.

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 6)): #Probamos entre 2 y 6 capas
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=100, #Cada uno de ellos tiene 100-300 neuronas, en intervalos de 100
                                            max_value=300,
                                            step=100),
                               activation='relu'))
    model.add(Dense(1, activation='linear')) #el último
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), #algunas tasas de aprendizaje
        loss='mse')
    return model

A continuación, especificamos cómo queremos explorar el espacio de parámetros. La búsqueda aleatoria es la opción más simple, pero a menudo bastante efectiva. Las alternativas son Hiperbanda (búsqueda aleatoria optimizada en la que se entrena una fracción mayor de modelos para un número menor de épocas, pero solo sobreviven las más prometedoras), o la optimización bayesiana, que intenta construir una interpretación probabilística de las notas del modelo (la probabilidad posterior de obtener una nota x, dados los valores de los hiperparámetros).

In [ ]:
tf.keras.backend.clear_session()

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=40, #cantidad de combinaciones para probar
    executions_per_trial=3,
    project_name='My Drive/Photoz') #es posible que se debe eliminar o restablecer en el futuro

Podemos visualizar el espacio de búsqueda así:

In [ ]:
tuner.search_space_summary()

Finalmente, es hora de poner nuestro sintonizador a trabajar. (¡Este es un gran trabajo!)

In [ ]:
tuner.search(Xst_train, y_train, #igual a model.fit
             epochs=100, validation_data=(Xst_val, y_val), batch_size=300, verbose = 0) 

#Nota: configurar la verbosidad en 0 no daría ningún resultado hasta que termine; tomó alrededor de ~ 35 minutos en mi computadora portátil

La función "resultados\_resumen(n)" nos da acceso a los n mejores modelos. Es útil mirar algunos porque a menudo las diferencias son mínimas y ¡podríamos preferir un modelo más pequeño! Tenga en cuenta que el parámetro "number of units" ("número de unidades) tendría un valor asignado para cada capa (incluso si el número de capas es menor en esa realización en particular).

In [ ]:
tuner.results_summary(5)

Las pérdidas de los primeros modelos son muy similares, lo que sugiere que 1. como de costumbre, necesitamos hacer algún tipo de validación cruzada para poder llegar a una clasificación, y 2. Con 3-5 capas y unos pocos cientos neuronas por capa, la configuración exacta no importa demasiado.

In [ ]:
best_hps=tuner.get_best_hyperparameters()[0] #elegir primer modelo

In [ ]:
best_hps.get('learning_rate')

In [ ]:
best_hps.get('num_layers')

In [ ]:
#Tamaño de las capas

print(best_hps.get('units_0'))
print(best_hps.get('units_1'))
print(best_hps.get('units_2'))

In [ ]:
model = tuner.hypermodel.build(best_hps) #obtener el mejor modelo

In [ ]:
model.build(input_shape=(None,6)) #construir el mejor modelo (si aún no se ajusta, esto le dará acceso al resumen)

In [ ]:
model.summary() #Mosca: esto es diferente a lo que vimos en el sumario del sintonizador

Ahora, construya una red neuronal con los hiperparámetros óptimos.

In [ ]:
bestnet = model.fit(Xst_train, y_train, validation_data= (Xst_val, y_val), epochs=100, batch_size=300)

También podemos mirar las curvas de entrenamiento vs validación para el modelo óptimo encontrado por el sintonizador.

In [ ]:
plt.plot(bestnet.history['loss'], label = 'train')
plt.plot(bestnet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.ylim(0,0.1)
plt.legend(loc='upper right', fontsize = 12)
plt.legend(fontsize = 12);
#plt.savefig('OptimalNN_Photoz.png',dpi=300)

Finalmente, informamos las notas de las pruebas para todas las métricas de interés (MSE, OLF, NMAD):

In [ ]:
model.evaluate(Xst_test, y_test)

In [ ]:
ypred = model.predict(Xst_test)

#Calcular FOL

print('OLF', len(np.where(np.abs(y_test-ypred)>0.15*(1+y_test))[0])/len(y_test))

#Calcular la desviación absoluta de la mediana normalizada (NMAD)

print('NMAD', 1.48*np.median(np.abs(y_test-ypred)/(1 + y_test)))

Estos números se mejoraron, en comparación con la versión de referencia; si la mejora es significativa o no, debe determinarse mediante la validación cruzada.

In [ ]:
plt.figure(figsize=(5,5))
    
plt.xlabel('True redshift', fontsize = 14)
plt.ylabel('Estimated redshift', fontsize = 14)

plt.scatter(y_test, model.predict(Xst_test), s =10, c = 'teal');

plt.xlim(0,2)
plt.ylim(0,2)
plt.tight_layout()
#plt.savefig('OptimalNN_scatter.png')

### A continuación, mostramos el efecto de cambiar la función de pérdida (MSE/MAE/MAPE), y estimamos las incertidumbres en las estimaciones de OLF/NMAD debido a la varianza de la muestra, para que podamos decidir si las diferencias son significativas.

#### El modelo es el mejor modelo que encontré arriba (provino de una búsqueda aleatoria, es posible que encuentre uno diferente).

In [ ]:
tf.keras.backend.clear_session()

model = keras.Sequential()

model.add(layers.Dense(units=300,
                               activation='relu'))
model.add(layers.Dense(units=200,
                               activation='relu'))
model.add(layers.Dense(units=100,
                               activation='relu'))
model.add(Dense(1, activation='linear')) #last one

#Usamos tres funciones de pérdida diferentes y repetimos el entrenamiento 4x

for loss in ['mse','mae', 'mape']:

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),
        loss=loss)

    OLF = np.zeros(4)
    NMAD = np.zeros(4)

    for i in range(0,4): #hagamos esto 4 veces y cambiemos solo la inicialización de pesos aleatorios
    
        model.fit(Xst_train, y_train,
             epochs=100,
             validation_data=(Xst_val, y_val), batch_size=300, verbose = 0)

        ypred = model.predict(Xst_test)

        #Calcular OLF

        OLF[i] = len(np.where(np.abs(y_test-ypred)>0.15*(1+y_test))[0])/len(y_test)

        #Calcular la desviación absoluta de la mediana normalizada (NMAD)
        
        NMAD[i] = 1.48*np.median(np.abs(y_test-ypred)/(1 + y_test))

    print('OLF mean/std using loss', loss, 'is:', "{:.3f}".format(OLF.mean()), "{:.3f}".format(OLF.std()))
    print('NMAD mean/std using loss', loss, 'is:', "{:.2f}".format(NMAD.mean()), "{:.3f}".format(NMAD.std()))

### Revisión de aprendizaje
    
¿Cuáles funciones de pérdida son las más adecuadas para minimizar el OLD y el NMAD?

<br>

<details>
<summary style="display: list-item;">Haga clic aquí para la respuesta!</summary>
<p>
    
```
Si queremos minimizar OLF/NMAD, nuestras opciones preferidas deberían ser las pérdidas MAE o MSE. En alternativa, podemos definir una pérdida personalizada.
```

</p>
</details>